In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

# Network and training.
EPOCHS = 200
BATCH_SIZE = 64
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.3

# Loading MNIST dataset.
# Labels have one-hot representation.
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
# X_train is 60000 rows of 28x28 values; we reshape it to 60000 x 784.
RESHAPED = 784
#

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize inputs within [0, 1].
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# One-hot representations for labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# Building the model.
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
          input_shape=(RESHAPED,),
          name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
          name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
          name='dense_layer_3', activation='softmax'))

# Summary of the model.
model.summary()

# Compiling the model.
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model.
model.fit(X_train, Y_train,
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
# Evaluating the model.
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)


In [ ]:
#create a file handle for the test data 
csv = open('digit_recogniser_test.csv')
next(csv)

In [ ]:
# A function that returns only one data element at a time from the dataset
def stream_data():
    s = next(csv)
    s = s.rstrip()
    l = s.split(',')
    l = [int(x)/255 for x in l]
    return np.array(l).reshape(1,784)

In [ ]:
# Loop over all values of the dataset and predict the number
predict = []
for _ in range(28000):
    prediction = model.predict(stream_data()).argmax()
    predict.append(prediction)

In [ ]:
# save the result as a csv file

import pandas as pd 

df = pd.DataFrame({'ImageId':list(range(1,28001)),'Label':predict})
df.to_csv('mnist_tensorflow.csv',index = False)

In [ ]:
# 0.99346 score on kaggle !